In [ ]:
# 드라이브 연동
# 딥러닝 폴더 접근 (경로 이동)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd 'drive/MyDrive/Colab Notebooks/23.12.18딥러닝'

/content/drive/MyDrive/Colab Notebooks/23.12.18딥러닝


### 목표
- mlp, cnn 모델 활용하여 이미지 분류 실습 진행
- 이미지 이진 분류 문제를 해결
- 성능을 높이기 위한 노력 진행
  - 이미지 증식(확장)
  - 전이학습

In [ ]:
# 환경 셋팅
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# npz 불러오기
data=np.load('./data/np_cats_vs_dogs.npz')
len(data)

4

In [ ]:
# 각 변수에 저장
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']


print("훈련셋",X_train.shape,y_train.shape)
print("테스트셋",X_test.shape,y_test.shape)


훈련셋 (2000, 224, 224, 3) (2000,)
테스트셋 (1000, 224, 224, 3) (1000,)


### mlp 모델 생성

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# 모델 설계
mlp_model= Sequential()

# 입력층(Flatten(데이터 크기= (행, 열, 채널수)))
mlp_model.add(Flatten(input_shape = (224,224,3)))

# 중간층 3(256,128,64), 활성화 = 'relu'
mlp_model.add(Dense(units=256,activation='relu'))
mlp_model.add(Dense(units=128,activation='relu'))
mlp_model.add(Dense(units=64,activation='relu'))

# 출력층 (이진 분류)
mlp_model.add(Dense(units=1,activation='sigmoid'))
# 유닛 개수=? 1 , 활성화=? 'sigmoid'


In [3]:
mlp_model.summary()

# 컴파일
# loss, optimizer, metrics
mlp_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
# 조기학습 중단 정의
# 교차검증 정확도를 기준으로 5번 정도 학습했을때도 갱신이 되지 않으면 멈춤
# 모니터 = 교차검증 정확도, 기다리는 횟수 = 5
f_ea =EarlyStopping(monitor='val_accuracy',# 모니터 할 값
                      patience=5, # 몇번 성능개선을 기다릴지 정하는 횟수
                      verbose=1) # 조기 학습중단 현황 문자열 출력

# 학습
# 검증 데이터 분리 30% , 반복 40, 배치 사이즈 64, 조기학습중단 연결
mlp_his=mlp_model.fit(X_train,y_train,
              validation_split=0.3,
              epochs=40,
              batch_size=64,
              callbacks=[f_ea])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 256)               38535424  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 38576641 (147.16 MB)
Trainable params: 38576641 (147.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


NameError: name 'X_train' is not defined

In [ ]:
# mlp_his 시각화
# accuracy, val_accuracy 선 그래프 변동 확인
plt.figure(figsize=(15,3))
plt.plot(mlp_his.history['accuracy'],label='acc')
plt.plot(mlp_his.history['val_accuracy'],label='val_acc')
plt.legend()
plt.show()

In [ ]:
# 모델이 학습을 제대로 하지 못한 상황
# 복잡한 데이터, 모델 단순한 상황
# mlp 특성상 이미지에 대한 학습 성능 떨어지고 있고, 그게 사실
# 이지미에 대한 파악을 잘 할 수 있게 해보자 > 중요한 특성 추출 학습 > CNN

#### CNN모델 생성
- 1. 모델 설계
  - 뼈대 구축
  - CONV(특성 추출부, 합성곱층)
  - MLP(분류부, 전결합층, 완전결합층)
- 2. 모델 컴파일(이진분류)
- 3. 학습 및 시각화
- 4. 예측 및 평가

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# 모델 설계
cnn_model= Sequential()
# 특성 추출부(Convolutional)
cnn_model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=X_train[0].shape, padding='same',activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

cnn_model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

cnn_model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))


# 분류부 (전결합층, mlp)
# mlp는 1차원만 받아들일 수 있음 Flatten
cnn_model.add(Flatten())
cnn_model.add(Dense(units=512,activation='relu'))


# 출력층 (이진 분류)
cnn_model.add(Dense(units=1,activation='sigmoid'))
# 유닛 개수=? 1 , 활성화=? 'sigmoid'

In [ ]:
cnn_model.summary()

# 컴파일
# loss, optimizer, metrics
cnn_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
# 조기학습 중단 정의
# 교차검증 정확도를 기준으로 5번 정도 학습했을때도 갱신이 되지 않으면 멈춤
# 모니터 = 교차검증 정확도, 기다리는 횟수 = 5
f_ea =EarlyStopping(monitor='val_accuracy',# 모니터 할 값
                      patience=5, # 몇번 성능개선을 기다릴지 정하는 횟수
                      verbose=1) # 조기 학습중단 현황 문자열 출력

# 학습
# 검증 데이터 분리 30% , 반복 40, 배치 사이즈 64, 조기학습중단 연결
cnn_his=cnn_model.fit(X_train,y_train,
              validation_split=0.3,
              epochs=40,
              batch_size=64,
              callbacks=[f_ea])


In [ ]:
plt.figure(figsize=(15,3))
plt.plot(cnn_his.history['accuracy'],label='acc')
plt.plot(cnn_his.history['val_accuracy'],label='val_acc')
plt.legend()
plt.show()


In [ ]:
# 성능이 mlp 보다는 학습이 잘 되고 있는 상황이지만, 과대적합이 되고 있는 상황
# train 에 대해 너무 맞춰서 학습하는 상황
# 과대적합을 해소하면서 모델의 일반화 성능을 더 높여야겠음

In [ ]:
# 환경설정
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# 데이터 로드
data = np.load("./data/np_cats_vs_dogs.npz")
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']


In [ ]:
# VGG16 모델 가져오기
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [ ]:
# VGG16은 16계층으로 이루어진 CNN 모델이다.
# ImageNet의 데이터베이스의 1백만개가 넘는 영상에 대해 사전 훈련된 모델
# 1000가지의 사물 범주를 분류해낼 수 있다.

In [ ]:
vgg16 = VGG16(include_top=False,
              weights = 'imagenet',
              input_shape = (224,224,3))
# include_top = False --> 불러온 모델의 MLP층을 사용하지 않겠다라는 뜻(비활성화)/ 특징추출부만 사용
# 기본값은 imageNet의 1000개 클래스에 대응하는 MLP 사용
# 우리는 강아지와 고양이만 분류하면 되기때문에 별도의 MLP를 사용할 예정
# weight ='imagent' : ImageNet에서 학습된 가중치(w)를 그대로 가져오겠다라는 뜻
# ImageNet은 1000여개의 클래스로 분류된 1백만개가 넘는 영상 이미지를 담고 있는 데이터 셋


In [ ]:
vgg16.summary()

In [ ]:
# VGG16모델에 층을 확인
for layer in vgg16.layers :
  print(layer.name) # vgg16모델의 층의 이름을 출력

In [ ]:
# 신경망 설계
transfer_model=Sequential()

# 전이학습 + 미세조정방식을 사용해보자
# vgg16모델에서 마지막 층을 학습 가능하도록 설정
# 그 외 나머지 층들은 학습이 되지 않도록 동결 시키자
for layer in vgg16.layers :
  # block5_conv3 층만 학습 가능하도록 설정
  if layer.name=='block5_conv3':
    layer.trainable=True

  # 학습이 불가능하도록 나머지 층들을 동결 시킴
  else :
    layer.trainable=False

transfer_model.add(vgg16)

# MLP 다층 퍼셉트론 분류기 만들어보기
transfer_model.add(Flatten()) # 분류기와 연결하기 위한 1차원 펴주기
transfer_model.add(Dense(units=128,activation='relu'))
transfer_model.add(Dense(units=1,activation='sigmoid'))

# 모델 요약
transfer_model.summary()

In [ ]:
# 모델 컴파일
# adam을 세부 조정하기 위해 별도로 불러왔다.
# from tensorflow.keras.optimizer import Adam
transfer_model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.0001),
                       metrics=['accuracy'])

In [ ]:
# 모델 학습
transfer_model.fit(X_train,y_train,
                   validation_split=0.3,
                   epochs=10)

In [ ]:
transfer_model.evaluate(X_test,y_test)

In [ ]:
# 주의점
# 한번 학습이 가능하도록 설정된 층들의 경우 가중치인 W값이 다 바뀌어서 저장되어 있기 때문에
# 동결층을 바꾸고 싶다면 VGG16 모델을 처음부터 import 해줘야 함


### 데이터 증강(이미지 증식)
- 과대적합이 일어나는 이유 중 하나는 훈련 데이터가 모자라기 때문이다.
- 훈련 데이터가 충분히 많다면 과대적합을 줄일 수 있다.
- 데이터 증강이란? 훈련 데이터를 유사하고 다양하게 변형하여 새로운 훈련 데이터처럼 추가적으로 사용하고자 하는 방법이며 훈련 데이터 수가 늘어난 효과를 얻고자 하는 방법이다.

In [ ]:
# 이미지 증식 도구 불러오기
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터 확장
train_gen = ImageDataGenerator( rescale=1./255, # 이미지의 스케일 조절(픽셀 0~1 사이로 조절)
                #  rotation_range=15, # 0도에서 360도 사이에서 이미지 회전
                #  width_shift_range=0.1, # 전체에서 10프로 내외 수평이동
                #  height_shift_range=0.1,# 전체에서 10프로 내외 수직이동
                #  shear_range=0.5, # 0.5라디안 내외 시계 반대방향으로 변형 (약 28.6도)
                #  zoom_range=[0.8,2.0], # 0.8, 2배로 축소 및 확대
                #  horizontal_flip=True, # 수평방향으로 뒤집기
                #  vertical_flip=True, # 수직방향으로 뒤집기
                #  fill_mode='nearest'  # 이미지의 빈공간을 채워주는 방법
                  )

test_gen=ImageDataGenerator(rescale=1./255)

# flow_from_directory() : 폴더 경로 지정
train_dir='./data/cats_and_dogs_filtered/train'
test_dir='./data/cats_and_dogs_filtered/test'

train_generator=train_gen.flow_from_directory( # 해당 경로로 찾아가서 이미지 증식을 하겠다.
                                              train_dir, # 폴더 경로 지정
                                              target_size=(224,224), # 변환할 이미지 사이즈 지정
                                              batch_size=10, # 한번에 변환할 이미지 수
                                              class_mode='binary' # 라벨 번호는 0, 폴더는 알파벳 순으로 읽어온다.

)
test_generator=test_gen.flow_from_directory( # 해당 경로로 찾아가서 이미지 증식을 하겠다.
                                              test_dir, # 폴더 경로 지정
                                              target_size=(224,224), # 변환할 이미지 사이즈 지정
                                              batch_size=10, # 한번에 변환할 이미지 수
                                              class_mode='binary' # 라벨 번호는 0, 폴더는 알파벳 순으로 읽어온다.

)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# 라벨링 결과 확인
print(train_generator.class_indices)
print(test_generator.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


In [ ]:
# 드롭아웃 import
from tensorflow.keras.layers import Dropout

cnn_model2 = Sequential()
cnn_model2.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(224, 224, 3)))
cnn_model2.add(MaxPooling2D(pool_size=2))

cnn_model2.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
cnn_model2.add(MaxPooling2D(pool_size=2))

cnn_model2.add(Conv2D(128, (3, 3), padding='same',activation='relu'))
cnn_model2.add(MaxPooling2D(pool_size=2))


cnn_model2.add(Flatten())
cnn_model2.add(Dense(64, activation='relu'))
cnn_model2.add(Dropout(0.35)) # 랜덤으로 35% 뉴런 학습 제외
cnn_model2.add(Dense(1, activation='sigmoid'))

In [ ]:
# 모델 컴파일
cnn_model2.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [ ]:
# 이미지 증식을 진행했을때, 학습 횟수를 늘려줘야함
# 학습이 진행될때, 결과가 들쭉날쭉하기 때문에 학습을 오래 시켜줄 필요가 있음
epoch=100
h2=cnn_model2.fit_generator(generator=train_generator,
                            epochs=epoch,
                            validation_data=test_generator)

In [ ]:
# 학습 결과 시각화
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
x_range = range(1,len(h2.history['accuracy'])+1)
plt.plot(x_range, h2.history['accuracy'], label='acc')
plt.plot(x_range,h2.history['val_accuracy'], ls = ':',marker = 'o',label='val_acc')
plt.legend()

plt.subplot(1,2,2)
plt.plot(x_range, h2.history['loss'], label='loss')
plt.plot(x_range,h2.history['val_loss'],ls = ':', marker = 'o',label='val_loss')
plt.legend()

plt.show()